In [ ]:
!pip install tld
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=a9df11d03318a0d34e1010c86f2a7b4d9938ab10407884b15eef38588e0dab60
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import requests,json,time,re,os,random,nltk
from bs4 import BeautifulSoup
from urllib.request import urljoin,urlparse
from collections import deque
from copy import deepcopy
from urllib.parse import urlparse
from tld import get_tld
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from langdetect import detect

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def mapper(i):
    if i=="benign":return 0
    else:return 1

startTime = time.time()
# dataset = pd.read_csv("malicious_phish.csv")
# dataset = pd.read_csv("/content/drive/MyDrive/Graph Based Cybersecurity Detection/Filtered_URLS.csv")
df = pd.read_csv("/content/drive/MyDrive/Graph Based Cybersecurity Detection/Filtered_URLS.csv")
df['type'] = df['type'].apply(lambda i: mapper(i))
# df.drop(['type'], axis = 1,inplace=True)
df2 = pd.read_csv("/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_URLS.csv")
dataset = df.drop(df2.index)
webpages = dict()
print(dataset.type.value_counts())
dataset.columns


0    21826
1     8688
Name: type, dtype: int64


Index(['url', 'type'], dtype='object')

In [ ]:
def crawl(url, maxdepth=3):
    visited = set()
    visited.add(url)
    edges = set()
    q = deque()
    q.append((url,None))
    depths = dict()
    depths[url] = 0
    cnt = 0
    while q:
        cnt += 1
        currUrl,sourceUrl = q.popleft()
        print(cnt, depths[currUrl])
        if depths[currUrl]>maxdepth:
            break
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',}
            getResponse = requests.get(currUrl,headers=headers,timeout=5)
        except:
            print("Error while accessing",currUrl)
            continue
        webpages[currUrl] = getResponse.content
        soup = BeautifulSoup(getResponse.content,"lxml")
        for anchor in soup.findAll("a"):
            if depths[currUrl]>maxdepth:
                break
            href = anchor.attrs.get("href")

            if(href != "" or href != None):
                href = urljoin(url, href)
                href_parsed = urlparse(href)
                href = href_parsed.scheme
                href += "://"
                href += href_parsed.netloc
                href += href_parsed.path
                final_parsed_href = urlparse(href)
                is_valid = bool(final_parsed_href.scheme) and bool(final_parsed_href.netloc)
                final_parsed_href = final_parsed_href.geturl()
                if is_valid and (final_parsed_href not in visited):
                    q.append((final_parsed_href,currUrl))
                    depths[final_parsed_href] = depths[currUrl]+1
                    edges.add((currUrl,final_parsed_href))
                    visited.add(final_parsed_href)

    return visited,edges

In [ ]:
def edgemapper(visited,edges):
    map = {visited[i]:i for i in range(len(visited))}
    newedges = []
    for i in edges:
        x,y = i
        newedges.append([map[x],map[y]])
    return newedges

In [ ]:
vis,edges = crawl("https://www.youtube.com")
mapping = dict()
idnum = 1
for i in vis:
    mapping[i] = idnum
    idnum+=1
newvis,newedges = [i for i in range(1,len(vis)+1)],set()
for i in edges:
    newedges.add((mapping[i[0]],mapping[i[1]]))
G = nx.DiGraph()
G.add_nodes_from(newvis)
G.add_edges_from(newedges)
url,utype = "https://www.youtube.com","benign"
newG = nx.convert_node_labels_to_integers(G)
oldNodes = list(G.nodes)
labels = {str(i):i for i in range(len(oldNodes))}
invlabels = {i:[i] for i in range(len(oldNodes))}
jsondict = dict()
jsondict["target"] = 0
jsondict["edges"] = [list(e) for e in newG.edges]
jsondict["labels"] = labels
jsondict["inverse_labels"] = invlabels
with open("youtube.json", "w") as outfile:
    outfile.write(json.dumps(jsondict))
nx.draw(G, with_labels = True)
plt.savefig("filename.png")
print(mapping)

In [ ]:
sampleSize = 1000
# sampleData = dataset.groupby('type', group_keys=False).apply(lambda x:x.sample(frac=(sampleSize/len(dataset)))) #proportional sampling
sampleData = dataset.groupby('type', group_keys=False).apply(lambda x:x.sample(sampleSize)) #Equal Sampling
# sampleData = dataset
outboundlinks = dict()
sampleData = sampleData.sample(frac=1)
print(sampleData)
print(sampleData.type.value_counts())

                                                     url  type
28908  http://www.versverwerktegroenten.nl/component/...     1
14960                 https://cospages.000webhostapp.com     1
8865   https://en.wikipedia.org/wiki/Wikipedia:AutoWi...     1
7392   http://fizzim.com/component/content/article/13...     1
19016  http://www.musimagen.com/index.php?option=com_...     1
...                                                  ...   ...
13805  https://allmusic.com/artist/macy-gray-p354914/...     0
11086  https://eventful.com/montreal/events/rogers-cu...     0
2099   https://f4ds4g6253dh737d648h7j6489.000webhosta...     1
4830      http://www.nissad.com/nissad-distributors.html     1
22519            https://medicine.yale.edu/labs/horwich/     0

[2000 rows x 2 columns]
1    1000
0    1000
Name: type, dtype: int64


In [ ]:
w2v_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Graph Based Cybersecurity Detection/GoogleNews-vectors-negative300.bin.gz', binary=True)

def w2v(inp, array_size):
    # Load pre-trained Word2Vec model
    # w2v_model = gensim.downloader.load("word2vec-google-news-300")
    tokens = word_tokenize(inp.lower())
    embeddings = [w2v_model[word] for word in tokens if word in w2v_model]
    if len(embeddings) == 0:
        return [-1] * array_size
    if len(embeddings) >= array_size:
        array_representation = np.mean(embeddings[:array_size], axis=0)
    else:
        padding = [np.zeros_like(embeddings[0])] * (array_size - len(embeddings))
        array_representation = np.mean(embeddings, axis=0)
        array_representation = np.concatenate((array_representation, padding))
    answer = [0]*array_size
    for i in range(len(array_representation)):
        answer[i%array_size] += array_representation[i]
    return answer


In [ ]:
def contentFeatures(url):
    fv = [0]*11
    outputArray = np.array(fv)

    languagemap = {'af': 0, 'ar': 1, 'bg': 2, 'bn': 3, 'ca': 4, 'cs': 5, 'cy': 6, 'da': 7, 'de': 8, 'el': 9, 'en': 10, 'es': 11, 'et': 12, 'fa': 13, 'fi': 14, 'fr': 15, 'gu': 16, 'he': 17, 'hi': 18, 'hr': 19, 'hu': 20, 'id': 21, 'it': 22, 'ja': 23, 'kn': 24, 'ko': 25, 'lt': 26, 'lv': 27, 'mk': 28, 'ml': 29, 'mr': 30, 'ne': 31, 'nl': 32, 'no': 33, 'pa': 34, 'pl': 35, 'pt': 36, 'ro': 37, 'ru': 38, 'sk': 39, 'sl': 40, 'so': 41, 'sq': 42, 'sv': 43, 'sw': 44, 'ta': 45, 'te': 46, 'th': 47, 'tl': 48, 'tr': 49, 'uk': 50, 'ur': 51, 'vi': 52, 'zh-cn': 53, 'zh-tw': 54}

    page = webpages.get(url,None)
    if not page:
        return np.array(list(outputArray) + [0,0,0,0,0])

    soup = BeautifulSoup(page,"lxml")
    pageContent = ' '.join([str(tag.text) for tag in soup.find_all() if tag!='script'])

    # HTML Tag Count
    outputArray[0] = len(soup.find_all())

    # Script tag count
    outputArray[1] = len(soup.find_all('script'))

    # External script tag count
    outputArray[2] = len(soup.find_all('script', {'src': True}))

    # Total Content Length
    outputArray[3] = len(pageContent)

    # Alphabet count and digit count
    for i in pageContent:
        if i.isalpha():
            outputArray[4]+=1
        elif i.isdigit():
            outputArray[5]+=1

    # Number of outbound links
    outputArray[6] = len(soup.find_all('a',{'href':True})) + len(soup.find_all('form', {'action': True}))

    # Content Language
    outputArray[7] = languagemap.get(detect(pageContent),-1)

    # Suspicious Word count
    suspiciousWordList = ["secure","update","account","login","signin","verify","payment","banking","bank","online","free","service","alert","lucky","notification","password","reset","offer","win","prize","discount","limited","exclusive","official"]
    for word in suspiciousWordList:
        if word in pageContent.lower():
            outputArray[8] += pageContent.lower().count(word)

    #Post request count
    outputArray[9] = len(soup.find_all('form', {'action': True}))

    # JS Code length
    scriptTags = soup.find_all('script')
    javascript_code_lens = [len(script.get_text()) for script in scriptTags]
    outputArray[10] = sum(javascript_code_lens)

    # Content w2v score

    outputArray = np.array(list(outputArray) + w2v(pageContent,5))

    return outputArray


In [ ]:
def URLFeatures(url):
    fv = [0]*21
    outputArray = np.array(fv)

    # URL Length
    outputArray[0] = len(url)

    # Alphabet and digit count
    for i in url:
        if i.isalpha():
            outputArray[1]+=1
        elif i.isdigit():
            outputArray[2]+=1

    # Special Character Counts
    specialChars = ['www','https','http','?','-','=','.','%','+']
    for i in range(len(specialChars)):
        outputArray[i+3] = url.count(specialChars[i])

    # Count redirects within URL
    pathobj = str(urlparse(url).path)
    outputArray[12] = pathobj.count('//')

    # Is the URL an IP Address
    parts = url.split(".")
    if parts[-1].isalpha():
        outputArray[13] = 0
    else:
        outputArray[13] = 1

    # Count of suspicious words
    suspiciousWordList = ["secure","update","account","login","signin","verify","payment","banking","bank","online","free","service","alert","lucky","notification","password","reset","offer","win","prize","discount","limited","exclusive","official"]
    for word in suspiciousWordList:
        if word in url.lower():
            outputArray[14] += 1

    # Is the URL shortened
    shorteningService = ['buzurl.com', 'twitthis.com', 'fic.kr', 'tinyurl', 'x.co', 'j.mp', 'short.to', 'yourls.org', 'go2l.ink', 'q.gs', 'v.gd', 'tweez.me', 'u.bb', 'om.ly', 'bitly.com', 'qr.net', 'rubyurl.com', 'vzturl.com', 'twit.ac', 'goo.gl', 'ff.im', 'tinyurl.com', 'qr.ae', 'bc.vc', 'Just.as', 'cli.gs', 'bit.ly', 'db.tt', 'adf.ly', 'twurl.nl', 'scrnch.me', 'ity.im', 't.co', 'bkite.com', 'cur.lv', 'u.to', 'ping.fm', 'goo.gl', 'filoops.info', 'bit.ly', 'loopt.us', 'ow.ly', 'ow.ly', 'tiny.cc', 'to.ly', 'short.ie', 't.co', 'bit.do', 'is.gd', '1url.com', 'post.ly', 'shorte.st', 'wp.me', 'link.zip.net', 'is.gd', 'kl.am', 'po.st', 'x.co', 'doiop.com', 'migre.me', 'su.pr', 'yfrog.com', 'url4.eu', 'lnkd.in', 'tr.im', 'snipr.com']
    for serv in shorteningService:
        if word in url.lower():
            outputArray[15] = 1
            break

    # Abnormal URL
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    outputArray[16] = 1 if match else 0

    # Hostname Length
    outputArray[17] = len(urlparse(url).netloc)

    # TLD Length
    _tld = get_tld(url,fail_silently=True)
    if _tld:
        outputArray[18] =  int(len(str(_tld)))
    else:
        outputArray[18] =  0

    # Directories
    outputArray[19] = pathobj.count('/')

    # URL w2v score
    outputArray[19] = sum(w2v(url,1))

    return outputArray

In [ ]:
# Node Feature Extraction

# URL Feature vector format:
# np.array([URL Length, Alphabet Count, Digit Count, 'www' Count,'https' Count,'http' Count,
#           '?' Count, '-' Count, '=' Count, '.' Count, '%' Count, '+' Count, Redirect Count, Is IP,
#           Suspicious Word Count, URL Shortened, Abnormal URL, Hostname Length, TLD Length, Directories in Path, URL w2v Score])
#
# Content Feature vector format:
# np.array([HTML Tag Count, Script Tag Count, External Script Count, Total Content Length, Alphabet Count, Digit Count, No. of Outbound Links, Content Language,
#           Suspicious Word Count, FormWithAction Count, JS Code Length, Content w2v score vector])


def extractFeatures(url):
    return np.array(list(contentFeatures(url)) + list(URLFeatures(url))).tolist()

x = extractFeatures("https://www.youtube.com")
print(x)


[323.0, 34.0, 9.0, 275717.0, 194652.0, 39053.0, 15.0, 10.0, 353.0, 0.0, 269676.0, -0.5411704775469843, -0.09074092330411077, -0.9517279763240367, 0.9176704348064959, 0.2718327974434942, 23.0, 18.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, -9.0, 0.0]


In [ ]:
len(x)

37

In [ ]:
def writeGraphToFile(nxgraph,folder,url,utype):
    utype = str(utype)
    filecount = len(os.listdir(folder))
    edgelist = [e for e in nxgraph.edges]
    nodemap,revmap = dict(),dict()
    nc = 0
    for i in nxgraph.nodes:
        nodemap[i] = nc
        revmap[nc] = i
        nc+=1
    namefile = open("/content/drive/MyDrive/Graph Based Cybersecurity Detection/filenames.txt","r")
    names = namefile.readlines()
    namefile.close()
    names = [i[:-1] for i in names]
    name = str(random.randint(1,10000))+".txt"
    while name in names:
        name = str(random.randint(1,10000))+".txt"
    namefile = open("/content/drive/MyDrive/Graph Based Cybersecurity Detection/filenames.txt","a")
    namefile.write(name+"\n")
    namefile.close()

    file = open(folder + "/" + name,"w")
    print(folder + "/" + name)
    file.write(url+"\n")
    file.write(utype+"\n")
    file.write(str(len(nxgraph.nodes)) + " " + str(len(nxgraph.edges)) + " " + str(len(nxgraph.nodes[revmap[0]]['feature'])) + "\n")
    for e in edgelist:
        i,j = e
        i,j = nodemap[i],nodemap[j]
        file.write(str(i) + " " + str(j) + "\n")
    for i in range(len(revmap)):
        urlnode = revmap[i]
        for j in nxgraph.nodes[urlnode]['feature']:
            file.write(str(j)+" ")
        file.write("\n")
    file.close()

In [ ]:
# graphData = dict()
egoNetsTrain,egoNetsTest = [],[]
for index, row in sampleData.iterrows():
    print("\n\n",index,"\n")
    temp = dict()
    url = row['url']
    if url[:4] != "http":
        url = "https://" + url
    urlType = row['type']
    temp['url'] = url
    temp['type'] = urlType
    visited,edges = crawl(url,10)
    G = nx.DiGraph()
    G.add_nodes_from(visited)
    G.add_edges_from(edges)
    if len(edges) == 0:continue
    temp['graph'] = nx.node_link_data(G)
    # graphData[url] = temp
    egoNetsTrain.append((G,url,urlType))
testsample = dataset.groupby('type', group_keys=False).apply(lambda x:x.sample(sampleSize//4))
for index, row in testsample.iterrows():
    temp = dict()
    url = row['url']
    if url[:4] != "http":
        url = "https://" + url
    urlType = row['type']
    temp['url'] = url
    temp['type'] = urlType
    visited,edges = crawl(url,10)
    G = nx.DiGraph()
    G.add_nodes_from(visited)
    G.add_edges_from(edges)
    if len(edges) == 0:continue
    temp['graph'] = nx.node_link_data(G)
    # graphData[url] = temp
    egoNetsTest.append((G,url,urlType))

# json_object = json.dumps(graphData, indent=4)
# with open("Graph_Data.json", "w") as outfile:
#     outfile.write(json_object)
print("Time of execution:", time.time()-startTime)


In [ ]:
egonets = egoNetsTrain + egoNetsTest
FOLDER_PATH = "/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs"
for i in egonets:
    graph,url,utype = i
    try:
        for n in graph.nodes:
            graph.nodes[n]['feature'] = extractFeatures(n)
    except:
        continue
    writeGraphToFile(graph,FOLDER_PATH,url,utype)
    filenew = open("/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_URLS.csv","a")
    filenew.write('"'+url+'"'+","+str(utype)+"\n")
    filenew.close()





# graph = deepcopy(egoNetsTrain[0][0])
# print([e for e in graph.nodes])
# [e for e in graph.edges]
# a = [np.array([i,i*i,i*i*i]) for i in range(10)]
# nx.set_node_attributes(graph, dict(zip(graph.nodes(), a)), name='feature')
# for n in graph.nodes:
#     print(graph.nodes[n]['feature'])

/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/8107.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/1942.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/2413.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/8461.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/3828.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/9006.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/9366.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/9025.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/9342.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/3257.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/6942.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/proces

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/5564.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/6056.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/1817.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/6295.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/4121.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/8203.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/6930.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/3566.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/3806.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/7274.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/processed_graphs/7499.txt
/content/drive/MyDrive/Graph Based Cybersecurity Detection/proces

In [ ]:
len(egonets)

2390